In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

## Load the merged GW dataset

Data retrieved from [https://github.com/vaastav/Fantasy-Premier-League](https://github.com/vaastav/Fantasy-Premier-League/tree/master/data/2023-24/gws)

Each row is the stats for a player in a given GW.  

In [12]:
prev_szn = '2023-24'
df = pd.read_csv(f'data/{prev_szn}_gws/merged_gw.csv')
df.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

Let's keep only relevant columns

In [13]:
df['GI'] = df['assists'] + df['goals_scored']
cols = ['name', 'position','team',
        'xP', 'expected_assists', 'expected_goals', 'expected_goal_involvements',
        'assists', 'goals_scored', 'GI',
        'bonus', 'bps',
        'minutes', 'total_points', 'value', 'GW']
df = df[cols]
df = df.rename(columns={'position': 'prev_position', 'expected_assists': 'xA', 'expected_goals': 'xG','expected_goal_involvements': 'xGI', 'assists': 'A', 'goals_scored': 'G', 'value': 'prev_price'})
df.head()

,name,prev_position,team,xP,xA,xG,xGI,A,G,GI,bonus,bps,minutes,total_points,prev_price,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0.00,0.00,0.00,0,0,0,0,0,0,0,40,1
1,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1
2,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0.05,0,0,0,0,4,22,1,70,1
3,Rhys Norrington-Davies,DEF,Sheffield Utd,0.1,0.00,0.00,0.00,0,0,0,0,0,0,0,40,1
4,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0.03,0,0,0,0,6,90,2,55,1


Let's remove the data from the relegated teams

In [14]:
# remove players from relegated teams
relegated = ['Sheffield Utd', 'Luton', 'Burnley']
df = df[~df['team'].isin(relegated)]
df.head()

,name,prev_position,team,xP,xA,xG,xGI,A,G,GI,bonus,bps,minutes,total_points,prev_price,GW
1,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1
2,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0.05,0,0,0,0,4,22,1,70,1
4,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0.03,0,0,0,0,6,90,2,55,1
5,Ionuț Radu,GK,Bournemouth,2.4,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1
7,Jack Grealish,MID,Man City,4.1,0.00,0.00,0.00,0,0,0,0,0,0,0,75,1


## Add the current season data to the df

Get data from the FPL API

In [15]:
# Get names and prices for the current szn
position_mapping = {
        1: "GK",
        2: "DEF",
        3: "MID",
        4: "FWD"
    }

team_mapping = {
    1: "Arsenal",
    2: "Aston Villa",
    3: "Bournemouth",
    4: "Brentford",
    5: "Brighton",
    6: "Chelsea",
    7: "Crystal Palace",
    8: "Everton",
    9: "Fulham",
    10: "Ipswich",
    11: "Leicester",
    12: "Liverpool",
    13: "Man City",
    14: "Man Utd",
    15: "Newcastle",
    16: "Nott'm Forest",
    17: "Southampton",
    18: "Spurs",
    19: "West Ham",
    20: "Wolves"
}
# Send a GET request to the current FPL API
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
data = response.json()

# Extract player information from the response
all_players = data['elements']

Get current season names, positions, and prices for all players

In [17]:
# Iterate over each player and print their name and price
names_current = []
positions_current = []
prices_current = []
team_current = []

for player in all_players:

    # get player name
    fname = player['first_name']
    lname = player['second_name']
    name = fname+' '+lname

    # get price at the start of the current szn
    price = player['now_cost'] - player['cost_change_start']

    # get current szn position - accounts for position changes
    element_type = player['element_type']
    position = position_mapping[element_type]

    # get current team
    team_id = player['team']
    team = team_mapping[team_id]

    #append to lists
    names_current.append(name)
    positions_current.append(position)
    prices_current.append(price)
    team_current.append(team)

# make sure each player has a name, position, and price
assert len(names_current) == len(positions_current) == len(prices_current) == len(team_current)

## Add current price and position to df

Get players from last season that are playing this season. 

Note: this does not account for name changes

In [18]:
# get names in both previous and current szn
names_prev = df['name'].unique()

ns = []
poss = []
ps = []
ts = []

# get names, positions, and prices for players in both previous and current szn
for i in range(len(names_current)):
    if names_current[i] in names_prev:
        ns.append(names_current[i])
        poss.append(positions_current[i])
        ps.append(prices_current[i])
        ts.append(team_current[i])
        
# name and price, and name and position mappings
name_price_mapping = dict(zip(ns, ps))
name_pos_mapping = dict(zip(ns, poss))
name_team_mapping = dict(zip(ns, ts))

# make sure mappings same size
assert len(name_price_mapping) == len(name_pos_mapping) == len(name_team_mapping), f"Mappings not same size: {len(name_price_mapping)} != {len(name_pos_mapping)}"

Add price and position from current szn to df

In [19]:
df['current_price'] = df['name'].map(name_price_mapping)
df['current_position'] = df['name'].map(name_pos_mapping)
df['current_team'] = df['name'].map(name_team_mapping)

df = df.dropna().reset_index(drop=True)
df.head()

,name,prev_position,team,xP,xA,xG,xGI,A,G,GI,bonus,bps,minutes,total_points,prev_price,GW,current_price,current_position,current_team
0,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,50.0,MID,Brighton
1,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0.05,0,0,0,0,4,22,1,70,1,65.0,MID,Man Utd
2,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0.03,0,0,0,0,6,90,2,55,1,50.0,MID,Brentford
3,Jack Grealish,MID,Man City,4.1,0.00,0.00,0.00,0,0,0,0,0,0,0,75,1,65.0,MID,Man City
4,Emil Krafth,DEF,Newcastle,0.0,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,45.0,DEF,Newcastle


In [20]:
df.head(50)

,name,prev_position,team,xP,xA,xG,xGI,A,G,GI,bonus,bps,minutes,total_points,prev_price,GW,current_price,current_position,current_team
0,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,50.0,MID,Brighton
1,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0.05,0,0,0,0,4,22,1,70,1,65.0,MID,Man Utd
2,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0.03,0,0,0,0,6,90,2,55,1,50.0,MID,Brentford
3,Jack Grealish,MID,Man City,4.1,0.00,0.00,0.00,0,0,0,0,0,0,0,75,1,65.0,MID,Man City
4,Emil Krafth,DEF,Newcastle,0.0,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,45.0,DEF,Newcastle
5,Boubacar Traoré,MID,Wolves,0.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,50.0,MID,Wolves
6,Curtis Jones,MID,Liverpool,2.1,0.01,0.00,0.01,0,0,0,0,1,24,1,50,1,55.0,MID,Liverpool
7,Reece James,DEF,Chelsea,2.1,0.42,0.03,0.45,0,0,0,0,15,75,2,55,1,50.0,DEF,Chelsea
8,Brandon Austin,GK,Spurs,1.0,0.00,0.00,0.00,0,0,0,0,0,0,0,40,1,40.0,GK,Spurs
9,Robin Olsen,GK,Aston Villa,0.5,0.00,0.00,0.00,0,0,0,0,0,0,0,40,1,45.0,GK,Aston Villa


In [12]:
print(f"There are {df['name'].nunique()} players in both the previous and current szn data")

There are 444 players in both the previous and current szn data


In [13]:
position_change_df = df[df['prev_position'] != df['current_position']]
position_change_df = position_change_df[position_change_df['GW']==1].reset_index(drop=True)
for i in range(len(position_change_df)):
    print(f"{position_change_df['name'].iloc[i]} changed positions from {position_change_df['prev_position'].iloc[i]} to {position_change_df['current_position'].iloc[i]}")

Richarlison de Andrade changed positions from MID to FWD
Kai Havertz changed positions from MID to FWD
Ian Maatsen changed positions from MID to DEF
Antoine Semenyo changed positions from FWD to MID
Christopher Nkunku changed positions from FWD to MID


In [14]:
save_dir = f'data/scouting'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

df.to_csv(f'{save_dir}/scouting_2024-25.csv', index=False)